In [156]:
import numpy as np
import os
from tqdm import tqdm
import itertools
import timeit
import torch

In [152]:
def optimized_gauss_integral(s1, e1, s2, e2):
    """
    Optimized calculation of the Gauss Link Integral (GLI) between two particle chains.

    Parameters:
        s1, e1, s2, e2: numpy.ndarray
            3D position vectors representing the start and end points of two chains.

    Returns:
        float: The computed GLI value.
    """
    # Compute vectors between points
    r13, r14 = s2 - s1, e2 - s1
    r23, r24 = s2 - e1, e2 - e1
    r12, r34 = e1 - s1, e2 - s2

    # Calculate face normals and their norms
    faces = [np.cross(r13, r14), np.cross(r14, r24), np.cross(r24, r23), np.cross(r23, r13)]
    norms = np.linalg.norm(faces, axis=1)

    # Normalize normals, handling zero norms gracefully
    normalized_faces = np.where(norms[:, None] != 0, faces / norms[:, None], np.zeros_like(faces))

    # Compute GLI using arcsin of dot products
    GLI = 0.0
    for i in range(4):
        dot = np.dot(normalized_faces[i], normalized_faces[(i + 1) % 4])
        dot = np.clip(dot, -1.0, 1.0)  # Clip to handle numerical issues
        GLI += np.arcsin(dot)

    # Determine the sign of GLI using cross-product and dot-product
    sign = np.dot(np.cross(r34, r12), r13)
    GLI *= -1 if sign <= 0 else 1

    # Scale by the normalization factor
    return GLI / (4.0 * np.pi)

# Example usage
def gauss_integral_all(path1, path2):
    """
    计算两个粒子路径之间的高斯链积分的总和。

    参数:
        path1: list of np.array, 粒子路径 1，每个元素是一个 [x, y, z] 的 NumPy 数组。
        path2: list of np.array, 粒子路径 2，每个元素是一个 [x, y, z] 的 NumPy 数组。
        gauss_integral_func: function, 用于计算两条链段之间的 GLI 的函数。
    返回:
        float: 高斯链积分总和。
    """

    GLI = 0.0

    # 遍历 path1 的每对连续粒子
    for i in range(len(path1) - 1):
        s1, e1 = path1[i], path1[i + 1]

        # 遍历 path2 的每对连续粒子
        for j in range(len(path2) - 1):
            s2, e2 = path2[j], path2[j + 1]

            # 调用 gauss_integral_func 计算当前两条链段的 GLI
            GLI += optimized_gauss_integral(s1, e1, s2, e2)

    return GLI

kinematic_chain = [[0, 2, 5, 8, 11], [0, 1, 4, 7, 10], [0, 3, 6, 9, 12, 15], [9, 14, 17, 19, 21], [9, 13, 16, 18, 20]]

class Skeleton(object):
    def __init__(self, kinematic_tree):
        self.kinematic_tree = kinematic_tree
        self.parents = [0] * 22
        self.parents[0] = -1
        self.children = [[] for _ in range(22)]
        for chain in self.kinematic_tree:
            for j in range(1, len(chain)):
                self.parents[chain[j]] = chain[j-1]
            for j in range(0, len(chain)-1):
                self.children[chain[j]].append(chain[j+1])
            

    def find_path_to_root(self, current):
        path = []
        while (current != -1):
            path.append(current)
            current = self.parents[current]  # 沿父节点方向走
        return path
    
    def find_path_between_leaves(self, start, end):
    # 获取从每个叶子节点到根节点的路径
        path1 = self.find_path_to_root(start)
        path2 = self.find_path_to_root(end)
        
        # 找到两个路径的公共部分（即最近公共祖先）
        set1 = set(path1)
        common_ancestor = 0
        for joint in path2:
            if joint in set1:
                common_ancestor = joint
                break
        
        # 构造从 leaf1 到 leaf2 的路径
        path_from_leaf1_to_ancestor = [joint for joint in path1[:path1.index(common_ancestor) + 1]]
        path_from_ancestor_to_leaf2 = [joint for joint in path2[:path2.index(common_ancestor)]][::-1]
        # 合并两个路径，得到从 leaf1 到 leaf2 的路径
        path_between_leaves = path_from_leaf1_to_ancestor + path_from_ancestor_to_leaf2
    
        return path_between_leaves


def gauss_integral_skeleton(paths, pose1, pose2):
    
    GLI = np.zeros([len(paths), len(paths)])
    for i in range(len(paths)):
        path1 = pose1[paths[i]]
        for j in range(len(paths)):
            path2 = pose2[paths[j]]
            GLI[i, j] = gauss_integral_all(path1, path2)
    return GLI
    
def gauss_integral_motion(kinematic_chain, motion1, motion2):
    # motion (frame_num, joint_num, 3)
    # skeleton = Skeleton(kinematic_chain)
    # end_points = [chain[-1] for chain in kinematic_chain]
    # combinations = list(itertools.combinations(end_points, 2))
    # combinations = [(a,b) if a<b else (b,a) for a,b in combinations]
    # paths = [skeleton.find_path_between_leaves(a,b) for a,b in combinations]
    paths = [chain[1:] for chain in kinematic_chain]
    paths_extra = [[2,14], [1,13]]
    paths = paths + paths_extra
    frame = min(len(motion1), len(motion2))
    GLI_motion = np.zeros([frame, len(paths), len(paths)])
    overlap_flags = np.zeros(frame, dtype=int)
    for i in range(frame):
        bbox1 = calculate_xz_bounding_box(motion1[i])
        bbox2 = calculate_xz_bounding_box(motion2[i])
        overlap_flags[i] = int(check_bounding_box_overlap(bbox1, bbox2))
        
    windows = []
    count = 0
    while count < frame:
        if overlap_flags[count] == 1:
            # 找到当前窗口的起点和终点，向前后扩展
            start = max(count - 1, 0)
            while count < frame and overlap_flags[count] == 1:
                count += 1
            end = min(count + 1 - 1, frame - 1)

            # 添加窗口范围
            windows.append((start, end))
        else:
            count += 1

    # 遍历所有帧，计算是否重合
    for start, end in windows:
        for i in range(start, end + 1):
            pose1 = motion1[i]
            pose2 = motion2[i]
            GLI_pose = gauss_integral_skeleton(paths, pose1, pose2)
            GLI_motion[i] = GLI_pose
    
    GLI_abs_vel = np.abs(GLI_motion[1:] - GLI_motion[:-1])
    GLI_abs_vel_max = np.max(GLI_abs_vel, axis=(1,2))
    return GLI_abs_vel_max



def calculate_xz_bounding_box(pose):
    x_coords = pose[:, 0]
    z_coords = pose[:, 2]
    min_x, max_x = np.min(x_coords), np.max(x_coords)
    min_z, max_z = np.min(z_coords), np.max(z_coords)
    return min_x, max_x, min_z, max_z

def check_bounding_box_overlap(bbox1, bbox2):
    min_x1, max_x1, min_z1, max_z1 = bbox1
    min_x2, max_x2, min_z2, max_z2 = bbox2

    # 检查是否在 x 或 z 方向完全分离
    if max_x1 < min_x2 or max_x2 < min_x1:
        return False  # x 方向分离
    if max_z1 < min_z2 or max_z2 < min_z1:
        return False  # z 方向分离

    return True

In [155]:
import time

motion1 = np.load('../results/multi/left_punch_control_th2_p0.npy')
motion2 = np.load('../results/multi/left_punch_control_th2_p2.npy')
time1 = time.time()
GLI = gauss_integral_motion(kinematic_chain, motion1, motion2)
time2 = time.time()
# GLI_new = gauss_integral_motion(kinematic_chain, motion1, motion2)
print(time2-time1)
print(np.where(GLI>0.4))
print(GLI[np.where(GLI>0.4)])

1.5511682033538818
(array([170, 172, 173, 176, 177, 180, 181, 182, 190], dtype=int64),)
[0.9148388  0.60379048 0.52057128 0.93691223 0.75647928 0.75609634
 0.96255628 0.61992333 0.4417685 ]


In [141]:
print(np.where(GLI>0.5))

(array([170, 172, 173, 176, 177, 181, 182], dtype=int64),)


In [74]:
print(np.where(GLI>0.8))

(array([170, 172, 175, 176, 177, 178, 181, 182, 190], dtype=int64),)


In [84]:
motion3 = np.load('../results/multi/left_punch_control_th2_p1.npy')
GLI_13 = gauss_integral_motion(kinematic_chain, motion1, motion3)
print(np.where(GLI_13>0.6))

100%|██████████| 210/210 [00:07<00:00, 27.67it/s]

(209,)
(array([], dtype=int64),)


In [164]:
def calculate_bounding_box(pose):
    # Compute bounding box along batch and frame dimensions
    x_coords, z_coords = pose[..., 0], pose[..., 2]
    return torch.min(x_coords, dim=-1).values, torch.max(x_coords, dim=-1).values, torch.min(z_coords, dim=-1).values, torch.max(z_coords, dim=-1).values

def check_overlap(bbox1, bbox2):
    # Check for overlap in batch across x and z dimensions
    min_x1, max_x1, min_z1, max_z1 = bbox1
    min_x2, max_x2, min_z2, max_z2 = bbox2
    overlap_x = (max_x1 >= min_x2) & (max_x2 >= min_x1)
    overlap_z = (max_z1 >= min_z2) & (max_z2 >= min_z1)
    return overlap_x & overlap_z

motion = np.zeros([2, 4])
pose = torch.rand([2,22,3])
pose2 = torch.rand([2,22,3])
check_overlap = check_overlap(calculate_bounding_box(pose), calculate_bounding_box(pose2))
print(check_overlap)
motion[:, 1] = check_overlap.int()
print(motion)

tensor([True, True])
[[0. 1. 0. 0.]
 [0. 1. 0. 0.]]


In [99]:
def gauss_integral_test():
    """
    Calculate the Gauss Link Integral (GLI) between two particle chains.

    Parameters:
        s1, e1, s2, e2: numpy.ndarray
            3D position vectors representing the start and end points of two chains.

    Returns:
        float: The computed GLI value.
    """
    s1, e1, s2, e2 = motion1[0, 0], motion1[0, 1], motion1[0, 4], motion1[0, 5]

    # Convert inputs to numpy arrays (if not already)
    # s1, e1, s2, e2 = map(np.asarray, [s1, e1, s2, e2])

    # Ensure points are unique
    # pos = [s1, e1, s2, e2]
    # for i in range(3):
    #     for j in range(i + 1, 4):
    #         if np.allclose(pos[i], pos[j]):
    #             return 0.0
    
    # Compute vectors between points
    r13 = s2 - s1
    r14 = e2 - s1
    r23 = s2 - e1
    r24 = e2 - e1

    # Calculate face normals
    n = [np.cross(r13, r14),
        np.cross(r14, r24),
        np.cross(r24, r23),
        np.cross(r23, r13)
    ]

    # Normalize normals
    n = [vec / np.linalg.norm(vec) if np.linalg.norm(vec) != 0 else np.zeros(3) for vec in n]

    # Compute GLI
    GLI = 0.0
    for i in range(4):
        dot = np.dot(n[i], n[(i + 1) % 4])
        dot = np.clip(dot, -1.0, 1.0)  # Clip to handle numerical issues
        GLI += np.arcsin(dot)

    # Determine the sign of GLI
    r12 = e1 - s1
    r34 = e2 - s2
    tmp = np.cross(r34, r12)
    dot = np.dot(tmp, r13)

    if dot <= 0:
        GLI *= -1

    return GLI / (4.0 * np.pi)


def optimized_gauss_integral_test():
    """
    Optimized calculation of the Gauss Link Integral (GLI) between two particle chains.

    Parameters:
        s1, e1, s2, e2: numpy.ndarray
            3D position vectors representing the start and end points of two chains.

    Returns:
        float: The computed GLI value.
    """
    # Compute vectors between points
    s1, e1, s2, e2 = motion1[0, 0], motion1[0, 1], motion1[0, 4], motion1[0, 5]
    
    r13, r14 = s2 - s1, e2 - s1
    r23, r24 = s2 - e1, e2 - e1
    r12, r34 = e1 - s1, e2 - s2

    # Calculate face normals and their norms
    faces = [np.cross(r13, r14), np.cross(r14, r24), np.cross(r24, r23), np.cross(r23, r13)]
    norms = np.linalg.norm(faces, axis=1)

    # Normalize normals, handling zero norms gracefully
    normalized_faces = np.where(norms[:, None] != 0, faces / norms[:, None], np.zeros_like(faces))

    # Compute GLI using arcsin of dot products
    GLI = 0.0
    for i in range(4):
        dot = np.dot(normalized_faces[i], normalized_faces[(i + 1) % 4])
        dot = np.clip(dot, -1.0, 1.0)  # Clip to handle numerical issues
        GLI += np.arcsin(dot)

    # Determine the sign of GLI using cross-product and dot-product
    sign = np.dot(np.cross(r34, r12), r13)
    GLI *= -1 if sign <= 0 else 1

    # Scale by the normalization factor
    return GLI / (4.0 * np.pi)


time1 = timeit.timeit(gauss_integral_test, number=100)
time2 = timeit.timeit(optimized_gauss_integral_test, number=100)
print(time1, time2)

0.017206699994858354 0.014556300011463463
